# Prepare the LandSat Image 
Convert from float64 to unit16 & nodata values are set to 0

Note to self the landsat directory currently being used is not the correct one. We need the one that was at the planet location..

In [1]:
import os
import rasterio
import numpy as np
from rasterio.plot import show
from coastseg_planet import processing
from coastseg_planet.processing import convert_from_float_to_unit16, create_tiled_raster,convert_landsat_to_model_format,get_best_landsat_from_dir,get_best_landsat_tifs
from skimage.io import imsave
from skimage import img_as_ubyte


## Get the Best LandSat Raster

In [2]:
# input the coastseg session with landsat data downloaded
downloaded_directory = r"C:\development\coastseg-planet\CoastSeg-Planet\landsat\santa_cruz_boardwalk\ID_1_datetime05-21-24__01_55_42" # this is a custom roi thats the same as the planet roi
# get the landsat with the lowest RMSE and the cloud cover mask
landsat_dir=get_best_landsat_from_dir(downloaded_directory)
landsat_path, landsat_cloud_mask_path=get_best_landsat_tifs(downloaded_directory)
print(f"The best satellite is {landsat_dir}")
print(f"The best landsat tiff is at : {landsat_path}")
print(f"The best landsat cloud mask is at : {landsat_cloud_mask_path}")

2
The best satellite is L9
The best landsat tiff is at : C:\development\coastseg-planet\CoastSeg-Planet\landsat\santa_cruz_boardwalk\ID_1_datetime05-21-24__01_55_42\L9\ms\2024-03-21-18-46-12_L9_ID_1_datetime05-21-24__01_55_42_ms.tif
The best landsat cloud mask is at : C:\development\coastseg-planet\CoastSeg-Planet\landsat\santa_cruz_boardwalk\ID_1_datetime05-21-24__01_55_42\L9\mask\2024-03-21-18-46-12_L9_ID_1_datetime05-21-24__01_55_42_mask.tif


## Convert the LandSat to RGB Tiff Model Compatible Format

In [3]:
landsat_processed_path  = processing.format_landsat_tiff(landsat_path)
print(f"The landsat image that will be used for the model is at : {landsat_processed_path}")

dtype: uint8, shape: (110, 168, 3), count: 3
Saving image to C:\development\coastseg-planet\CoastSeg-Planet\landsat\santa_cruz_boardwalk\ID_1_datetime05-21-24__01_55_42\L9\ms\2024-03-21-18-46-12_L9_ID_1_datetime05-21-24__01_55_42_processed.jpg
The landsat image that will be used for the model is at : C:\development\coastseg-planet\CoastSeg-Planet\landsat\santa_cruz_boardwalk\ID_1_datetime05-21-24__01_55_42\L9\ms\2024-03-21-18-46-12_L9_ID_1_datetime05-21-24__01_55_42_processed_model_format.tif


# Filter the Planet Images
- Remove images that are less than 80% of the expected area
- todo: Remove images that are too cloudy
- todo use Mark's good bad classifier to remove bad images

In [4]:
import glob
import os
from coastseg_planet.processing import filter_tiffs_by_area
planet_dir = r"C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files"
tiff_paths = glob.glob(os.path.join(planet_dir,"*AnalyticMS_clip.tif"))
print(f"Number of TIFFs: {len(tiff_paths)}")

# improvement get the planet tiff with the largest area and use that as the expected area
expected_area_km = 4.5  # 4.1 square kilometers
threshold = 0.90

filtered_tiffs = filter_tiffs_by_area(tiff_paths, expected_area_km, threshold, use_threads=True)
print(f"Number of filtered TIFFs: {len(filtered_tiffs)}")
print("Filtered TIFFs:", filtered_tiffs)

Number of TIFFs: 23
Number of filtered TIFFs: 21
Filtered TIFFs: ['C:\\development\\coastseg-planet\\CoastSeg-Planet\\santa_cruz_boardwalk_QGIS\\files\\20240317_180741_09_2460_3B_AnalyticMS_clip.tif', 'C:\\development\\coastseg-planet\\CoastSeg-Planet\\santa_cruz_boardwalk_QGIS\\files\\20240401_180836_68_24b4_3B_AnalyticMS_clip.tif', 'C:\\development\\coastseg-planet\\CoastSeg-Planet\\santa_cruz_boardwalk_QGIS\\files\\20240319_190343_01_24fb_3B_AnalyticMS_clip.tif', 'C:\\development\\coastseg-planet\\CoastSeg-Planet\\santa_cruz_boardwalk_QGIS\\files\\20240409_180644_86_24b6_3B_AnalyticMS_clip.tif', 'C:\\development\\coastseg-planet\\CoastSeg-Planet\\santa_cruz_boardwalk_QGIS\\files\\20240317_181214_28_2459_3B_AnalyticMS_clip.tif', 'C:\\development\\coastseg-planet\\CoastSeg-Planet\\santa_cruz_boardwalk_QGIS\\files\\20240318_185725_57_247d_3B_AnalyticMS_clip.tif', 'C:\\development\\coastseg-planet\\CoastSeg-Planet\\santa_cruz_boardwalk_QGIS\\files\\20240317_180743_00_2460_3B_AnalyticMS_

# Convert the Planet Images to TOAR if not already TOAR
-todo read the xml files to determine if the TOAR tool was already used and if so skip this

In [5]:
from coastseg_planet.processing import convert_planet_to_model_format
from coastseg_planet.processing import get_base_filename
from coastseg_planet.processing import TOA_conversion

separator='_3B'
for target_path in filtered_tiffs:
    base_filename = get_base_filename(target_path, separator)
    output_path = os.path.join(planet_dir, f"{base_filename}_TOAR.tif")
    found_files = glob.glob(os.path.join(planet_dir, f"{base_filename}*AnalyticMS_metadata_clip.xml"))
    if not found_files:
        print(f"Could not find XML file for {target_path}")
        continue
    xml_path = found_files[0]
    print(f"Converting {target_path} to TOAR format and saving to {output_path}")
    # print(f"Using XML file: {xml_path}")
    TOA_conversion(target_path, xml_path, output_path)
    
    
# @todo remove the temp file

Converting C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240317_180741_09_2460_3B_AnalyticMS_clip.tif to TOAR format and saving to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240317_180741_09_2460_TOAR.tif
Sensor: PSB.SD
Saving to: C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240317_180741_09_2460_TOAR.tif_TOA.tif
Converting C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240401_180836_68_24b4_3B_AnalyticMS_clip.tif to TOAR format and saving to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240401_180836_68_24b4_TOAR.tif
Sensor: PSB.SD
Saving to: C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240401_180836_68_24b4_TOAR.tif_TOA.tif
Converting C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240319_190343_01_24fb_3B_AnalyticMS_clip.tif to TOAR format 

# Convert Planet Imagery to Same Format as LandSat
- reorder to bands to be  RGBNIR
- scale the values to be 0 - 255 

In [6]:
# from coastseg_planet.processing import convert_planet_to_model_format
from coastseg_planet.processing import get_base_filename
from coastseg_planet.processing import convert_planet_to_model_format

import numpy as np
from scipy.ndimage import zoom


separator='_3B'
for target_path in filtered_tiffs:
    base_filename = get_base_filename(target_path, separator)
    # make sure to use the TOAR file
    target_pat = os.path.join(planet_dir, f"{base_filename}_TOAR.tif")
    # make 
    output_path = os.path.join(planet_dir, f"{base_filename}_TOAR_model_format.tif")
    output_path= convert_planet_to_model_format(target_path,output_path)
    print(f"Converting to model format and saving to {output_path}")

Converting to model format and saving to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240317_180741_09_2460_TOAR_model_format.tif
Converting to model format and saving to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240401_180836_68_24b4_TOAR_model_format.tif
Converting to model format and saving to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240319_190343_01_24fb_TOAR_model_format.tif
Converting to model format and saving to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240409_180644_86_24b6_TOAR_model_format.tif
Converting to model format and saving to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240317_181214_28_2459_TOAR_model_format.tif
Converting to model format and saving to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240318_185725_57_247d_TOAR_model_format.tif
Conv

## Filter the Planet Imagery
Use a model to sort the images as good or bad

In [7]:
from coastseg_planet import model
import os 

planet_dir = r"C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files"
good_dir = os.path.join(planet_dir, "good")

model_path = r"C:\development\coastseg-planet\CoastSeg-Planet\models\best_rgb.h5"
path_to_inference_imgs = r"C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files"
output_folder = r"C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files"
csv_path = os.path.join(output_folder, "classification_results.csv")
regex = '*TOAR_model_format'
# model.run_classification_model(model_path, path_to_inference_imgs, output_folder, csv_path, regex, move_files=False)

Transformers is only compatible with Keras 2, but you have explicitly set `TF_USE_LEGACY_KERAS` to `0`. This may result in unexpected behaviour or errors if Keras 3 objects are passed to Transformers models.


Version:  2.15.0
Eager mode:  True


# Apply Cloud Masks to Planet
Create a cloud and shadow mask for each image in the directory of planet images

In [8]:
from coastseg_planet.masking import apply_cloudmask_to_dir
# apply cloud mask to directory of planet images and generate cloud masks in that directory for each tif
planet_dir = r"C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files"
good_dir = os.path.join(planet_dir, "good")
apply_cloudmask_to_dir(good_dir)

Processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240313_180824_31_2423_3B_udm2_clip.tif
Creating cloud and shadow mask from C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240313_180824_31_2423_3B_udm2_clip.tif
Saving combined mask to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240313_180824_31_2423_3B_udm2_clip_combined_mask.tif
Processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240313_181345_71_242d_3B_udm2_clip.tif
Creating cloud and shadow mask from C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240313_181345_71_242d_3B_udm2_clip.tif
Saving combined mask to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240313_181345_71_242d_3B_udm2_clip_combined_mask.tif
Processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_Q

# Coregister the Planet Imagery to the LandSat Image
- We will coregister each modified planet image to the landsat image that was modified to be in the zoo format

In [9]:
from coastseg_planet.coregister import global_coregister_gw,get_cpus,coregister_arosics_local
from coastseg_planet.processing import get_base_filename
# try to coregister two planet images together that have clouds now that we can mask out clouds

kwargs = {
   'max_shift':30,
   'align_grids':True,
   'grid_res':30, # Grid points are 30 meters apart (the lower this value the longer it takes)
   'v':True, #verbose mode
   'q':False, #quiet mode
   'ignore_errors':False, # Recommended to set to True for batch processing
    'progress':True,
    'out_crea_options':["COMPRESS=LZW"], # otherwise if will use deflate which will not work well with our model
    'fmt_out':'GTIFF',
    'CPUs' : get_cpus()/2,
}

separator='_3B'
for target_path in filtered_tiffs:
    print(f"Processing {target_path}")
    base_filename = get_base_filename(target_path, separator)
    # C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240317_181214_28_2459_TOAR_model_format.tif
    input_path = os.path.join(good_dir, f"{base_filename}_TOAR_model_format.tif") # this is the planet image we want to coregister
    if os.path.exists(input_path) == False:
        print(f"Could not find {input_path}")
        continue
    # add a function to do this in case the cloud mask is not found
    cloud_maks_found =  glob.glob(os.path.join(good_dir, f"{base_filename}*udm2_clip_combined_mask.tif")) # this is the cloud mask for the planet image
    if cloud_maks_found:
        target_cloud_mask_path = cloud_maks_found[0]
    outfile_path = os.path.join(good_dir, f"{ base_filename}{separator}_TOAR_processed_coregistered.tif") # this is the output path for the coregistered planet image
    
    # @todo remove the intermediate model format file
    
    # add the reference cloud mask
    
    try:
        # coregistered_target_path = coregister_arosics_local(input_path,landsat_processed_path, outfile_path, target_cloud_mask = target_cloud_mask_path,**kwargs) 
        coregistered_target_path = coregister_arosics_local(input_path,landsat_processed_path,landsat_cloud_mask_path, outfile_path, target_cloud_mask = target_cloud_mask_path,**kwargs) 
    except Exception as e:
        print(f"Error processing {target_path}: {e}")
        continue
    print(f"Coregistered planet image saved to: {coregistered_target_path}")

Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00


Number of CPUs available: 20
Processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240317_180741_09_2460_3B_AnalyticMS_clip.tif
Number of CPUs available: 20
{'max_shift': 30, 'align_grids': True, 'grid_res': 30, 'v': True, 'q': False, 'ignore_errors': False, 'progress': True, 'out_crea_options': ['COMPRESS=LZW'], 'fmt_out': 'GTIFF', 'CPUs': 10.0}
Calculating footprint polygon and actual data corner coordinates for reference image...
Bounding box of calculated footprint for reference image:
	(586582.5, 4090252.5, 589087.5, 4091902.5)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...
Bounding box of calculated footprint for image to be shifted:
	(586587.0, 4090254.0, 589095.0, 4091901.0)
resolutions:  15.0 3.0
99.92 percent of the image to be shifted is covered by the reference image.
Matching window position (X,Y): 587840.6848061499/4091078.3293753574
Target window size (256, 256) not possible due to too sma

Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |--------------------------------------------------| 0.1% Complete  => 0:00:00

Calculating footprint polygon and actual data corner coordinates for reference image...
Bounding box of calculated footprint for reference image:
	(586582.5, 4090252.5, 589087.5, 4091902.5)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...
Bounding box of calculated footprint for image to be shifted:
	(586587.0, 4090254.0, 589095.0, 4091901.0)
resolutions:  15.0 3.0
99.92 percent of the image to be shifted is covered by the reference image.
Matching window position (X,Y): 587840.6848061499/4091078.3293753574
Target window size (256, 256) not possible due to too small overlap area or window position too close to an image edge. New matching window size: (106, 106).
Initializing tie points grid...
Error processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240401_180836_68_24b4_3B_AnalyticMS_clip.tif: The bad data mask must have only two pixel values (boolean) - 0 and 1 or False and True! The given mask for 20

Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |==============------------------------------------| 27.2% Complete  => 0:00:00

...

Initializing tie points grid...
Error processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240401_190203_53_24f2_3B_AnalyticMS_clip.tif: The bad data mask must have only two pixel values (boolean) - 0 and 1 or False and True! The given mask for 2024-03-21-18-46-12_L9_ID_1_datetime05-21-24__01_55_42_mask contains the values [0, 21824, 21890, 21952, 22018, 22080, 22280, 23826, 23888, 24082, 24144].
Processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240409_190216_26_24fc_3B_AnalyticMS_clip.tif
Number of CPUs available: 20
{'max_shift': 30, 'align_grids': True, 'grid_res': 30, 'v': True, 'q': False, 'ignore_errors': False, 'progress': True, 'out_crea_options': ['COMPRESS=LZW'], 'fmt_out': 'GTIFF', 'CPUs': 10.0}
Calculating footprint polygon and actual data corner coordinates for reference image...
Bounding box of calculated footprint for reference image:
	(586582.5, 4090252.5, 589087.5, 4091902.5)
Calculating f

Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |=========================================---------| 82.0% Complete  => 0:00:00

Bounding box of calculated footprint for reference image:
	(586582.5, 4090252.5, 589087.5, 4091902.5)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...
Bounding box of calculated footprint for image to be shifted:
	(586587.0, 4090254.0, 589095.0, 4091901.0)
resolutions:  15.0 3.0
99.92 percent of the image to be shifted is covered by the reference image.
Matching window position (X,Y): 587840.6848061499/4091078.3293753574
Target window size (256, 256) not possible due to too small overlap area or window position too close to an image edge. New matching window size: (106, 106).
Initializing tie points grid...
Error processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240410_185931_91_24f3_3B_AnalyticMS_clip.tif: The bad data mask must have only two pixel values (boolean) - 0 and 1 or False and True! The given mask for 2024-03-21-18-46-12_L9_ID_1_datetime05-21-24__01_55_42_mask contains the values [0, 21824,

Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00


Bounding box of calculated footprint for reference image:
	(586582.5, 4090252.5, 589087.5, 4091902.5)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...
Bounding box of calculated footprint for image to be shifted:
	(586587.0, 4090254.0, 589095.0, 4091901.0)
resolutions:  15.0 3.0
99.92 percent of the image to be shifted is covered by the reference image.
Matching window position (X,Y): 587840.6848061499/4091078.3293753574
Target window size (256, 256) not possible due to too small overlap area or window position too close to an image edge. New matching window size: (106, 106).
Initializing tie points grid...
Error processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\20240411_190131_50_24e1_3B_AnalyticMS_clip.tif: The bad data mask must have only two pixel values (boolean) - 0 and 1 or False and True! The given mask for 2024-03-21-18-46-12_L9_ID_1_datetime05-21-24__01_55_42_mask contains the values [0, 21824,

# Apply the Model to the Planet Imagery

## Create an Elevation Mask from the Topobathy Data
- We will use this as our reference shoreline buffer
- This code is a bit of a placeholder eventually we will use the code to download the topobathy for the region 
- For now manually provide the topobathy tiff

In [10]:
topobathy_tiff = r"C:\development\coastseg-planet\CoastSeg-Planet\SantaCruzBoardwalk_topobathy.tif"

In [11]:
import rasterio
from ipyleaflet import Map, ImageOverlay
from ipywidgets import  VBox, FloatRangeSlider
import numpy as np
import PIL.Image
from io import BytesIO
import base64

# Read the TIFF file
tiff_file = r"C:\development\coastseg-planet\CoastSeg-Planet\Santa Cruz Boardwalk_topobathy.tif"
# tiff_file = r"C:\development\coastseg-planet\CoastSeg-Planet\Madeira_topobathy.tif"
with rasterio.open(tiff_file) as src:
    mask_data = src.read(1)  # Read the first band
    bounds = src.bounds
    transform = src.transform

# Function to create a base64 PNG image from mask data with transparency
def create_masked_image(mask_data, alpha=128):
    # Normalize the mask data to 0-255
    mask_data = (mask_data / mask_data.max() * 255).astype(np.uint8)
    # Create an RGBA image with the red channel filled and alpha channel set by the mask
    rgba_image = np.zeros((mask_data.shape[0], mask_data.shape[1], 4), dtype=np.uint8)
    rgba_image[..., 0] = mask_data  # Red channel
    # Set alpha to the specified value for the masked regions, 0 for the unmasked regions
    rgba_image[..., 3] = np.where(mask_data > 0, alpha, 0)  # Alpha channel (transparency)
    img = PIL.Image.fromarray(rgba_image)
    with BytesIO() as buffer:
        img.save(buffer, format="png")
        return base64.b64encode(buffer.getvalue()).decode('utf-8')

# Create the initial masked image overlay
mask_data = np.flipud(mask_data)  # Flip the mask data vertically
masked_image = create_masked_image(mask_data)
image_overlay = ImageOverlay(
    url="data:image/png;base64," + masked_image,
    bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]]
)

# Create the map
m = Map(center=[(bounds.top + bounds.bottom) / 2, (bounds.left + bounds.right) / 2], zoom=15)
m.add_layer(image_overlay)

# Function to update the mask (simulated here for demonstration purposes)
def update_mask(change):
    global image_overlay
    if not isinstance(change, tuple):
        mask = mask_data < change
        masked_image = create_masked_image(mask)
        image_overlay.url = "data:image/png;base64," + masked_image
    else:
        lower, upper = change
        mask = np.logical_and(mask_data > lower, mask_data < upper)
        masked_image = create_masked_image(mask)
        image_overlay.url = "data:image/png;base64," + masked_image


default_value = (-10, 10)
# Create a bounded slider with two handles
threshold_slider = FloatRangeSlider(
    description='Elevation (m)',
    value=[default_value[0], default_value[1]],
    min=mask_data.min(),
    max=mask_data.max(),
    step=0.1,
)

# Link the slider to the update function
threshold_slider.observe(lambda change: update_mask(change['new']), names='value')

# Display the map and the slider
display(VBox([m, threshold_slider]))

# Initial display
update_mask(default_value)

In [14]:
from coastseg_planet.processing import create_elevation_mask
from coastseg_planet.processing import match_resolution_and_grid,reproject_to_utm

# defaults if you don't want to use the slider for this example
# -2.86,16.032718467712403
low_threshold = threshold_slider.value[0]
high_threshold = threshold_slider.value[1]

masked_tif_path = create_elevation_mask(topobathy_tiff,topobathy_tiff.replace(".tif","_mask.tif"),low_threshold,high_threshold)
print(f"masked_tif_path: {masked_tif_path}")
# convert the mask to the UTM projection (this is the same as the landsat image)
output_path = reproject_to_utm(masked_tif_path, masked_tif_path.replace('.tif', '_utm.tif'))
print(f"output_path: {output_path}")

masked_tif_path: C:\development\coastseg-planet\CoastSeg-Planet\SantaCruzBoardwalk_topobathy_mask.tif
output_path: C:\development\coastseg-planet\CoastSeg-Planet\SantaCruzBoardwalk_topobathy_mask_utm.tif


In [15]:
from coastseg_planet.shoreline_extraction import get_shorelines_from_model
from coastseg_planet.processing import get_date_from_path,TOA_conversion,get_epsg_from_tiff,get_base_filename,convert_planet_to_model_format
from datetime import datetime
from coastseg_planet.model import apply_model_to_image
from coastseg.extracted_shoreline import load_image_labels, load_merged_image_labels, remove_small_objects_and_binarize, get_indices_of_classnames, get_class_mapping
from coastseg import file_utilities

# Future improvement will download & load the model from the coastseg_planet package
# for now hard code the model path & weights directory
model_card_path = r'C:\development\coastseg-planet\CoastSeg-Planet\output_zoo\coastseg_planet\coastseg_planet\config\model_card.yml'
weights_directory = r'C:\development\doodleverse\coastseg\CoastSeg\src\coastseg\downloaded_models\segformer_RGB_4class_8190958'
model_card_path = file_utilities.find_file_by_regex(
    weights_directory, r".*modelcard\.json$"
)
# all this is to create the settings dictionary needed to extract shorelines
out_epsg = get_epsg_from_tiff(landsat_processed_path)
# convert str to int
out_epsg = int(out_epsg)
settings = {
    'output_epsg': out_epsg,
    'min_length_sl': 100,
    'dist_clouds': 50,
    'min_beach_area': 1000,
    'satname': 'planet',
}

separator = '_3B'
count = 0
for target_path in filtered_tiffs:
    count += 1
    base_filename = get_base_filename(target_path, separator)
    print(f"Processing {base_filename}")
    # get the processed coregistered file, the cloud mask and the npz file
    input_path = os.path.join(good_dir, f"{ base_filename}{separator}_TOAR_processed_coregistered.tif") # this is the input path for the coregistered planet image
    if not os.path.exists(input_path):
        print(f"Skipping {input_path} because it does not exist")
        continue
    found_cloud_masks =  glob.glob(os.path.join(good_dir, f"{base_filename}*udm2_clip_combined_mask.tif"))
    if not found_cloud_masks:
        print(f"Skipping {input_path} because cloud mask not found")
        continue
    cloud_mask_path = found_cloud_masks[0]
    date=get_date_from_path(base_filename)
    print(f"Processing {input_path}")
    print(f"Using cloud mask: {cloud_mask_path}")
    if not os.path.exists(input_path):
        print(f"Skipping {input_path} because it does not exist")
        continue
    
    
    apply_model_to_image(input_path,good_dir,False,False)
    # this is the file generated by the model
    npz_path = os.path.join(good_dir, 'out', f"{base_filename}_3B_TOAR_processed_coregistered_res.npz")
    if not os.path.exists(npz_path):
        print(f"Skipping {npz_path} because it does not exist")
        

    output_tiff =  r"C:\development\coastseg-planet\CoastSeg-Planet\Santa_Cruz_Boardwalk_topobathy_masked_matched_resolution_.tif"
    output_tiff= output_tiff.replace('.tif', f'_{count}.tif')
    match_resolution_and_grid(output_path, input_path, output_tiff)
    
    save_path = os.path.join(good_dir, 'out')
    shoreline = get_shorelines_from_model(cloud_mask_path,input_path,model_card_path,npz_path,date,satname= 'planet', settings = settings,topobathy_path = output_tiff,
                                          save_path=save_path)
    


32610
Processing 20240317_180741_09_2460
Processing C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240317_180741_09_2460_3B_TOAR_processed_coregistered.tif
Using cloud mask: C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240317_180741_09_2460_3B_udm2_clip_combined_mask.tif



Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Processing image:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240317_180741_09_2460_3B_TOAR_processed_coregistered.tif
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240317_180741_09_2460_3B_TOAR_processed_coregistered_predseg.png
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240317_180741_09_2460_3B_TOAR_processed_coregistered_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3

saving segmentation results to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240317_180741_09_2460_3B_TOAR_processed_coregistered_predseg.png
writing metadata to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240317_180741_09_2460_3B_TOAR_processed_coregistered_res.npz
32610
image_epsg: 32610 type: <class 'int'>
4.242640687119285
58.24264068711

Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing image:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240319_190343_01_24fb_3B_TOAR_processed_coregistered.tif
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240319_190343_01_24fb_3B_TOAR_processed_coregistered_predseg.png
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240319_190343_01_24fb_3B_TOAR_processed_coregistered_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240319_190343_01_24fb_3B_TOAR_processed_coregistered_predseg.png
writing metadata to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240319_190343_01_24fb_3B_TOAR_processed_coregistered_res.npz
32610
image_epsg: 32610 type: <class 'int'>
2845.6967517062512
1627.1864709697

...

Processing image:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240402_182010_48_2432_3B_TOAR_processed_coregistered.tif
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240402_182010_48_2432_3B_TOAR_processed_coregistered_predseg.png
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240402_182010_48_2432_3B_TOAR_processed_coregistered_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240402_182010_48_2432_3B_TOAR_processed_coregistered_predseg.png
writing metadata to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240402_182010_48_2432_3B_TOAR_processed_coregistered_res.npz
32610
image_epsg: 32610 type: <class 'int'>
2602.1088732275775
358.27922061357

Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing image:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240313_180824_31_2423_3B_TOAR_processed_coregistered.tif
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240313_180824_31_2423_3B_TOAR_processed_coregistered_predseg.png
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240313_180824_31_2423_3B_TOAR_processed_coregistered_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240313_180824_31_2423_3B_TOAR_processed_coregistered_predseg.png
writing metadata to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240313_180824_31_2423_3B_TOAR_processed_coregistered_res.npz
32610
image_epsg: 32610 type: <class 'int'>
433.9188309203679
201.338095116624

Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing image:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240401_190203_53_24f2_3B_TOAR_processed_coregistered.tif
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\20240401_190203_53_24f2_3B_TOAR_processed_coregistered_predseg.png
Output file:  C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240401_190203_53_24f2_3B_TOAR_processed_coregistered_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240401_190203_53_24f2_3B_TOAR_processed_coregistered_predseg.png
writing metadata to C:\development\coastseg-planet\CoastSeg-Planet\santa_cruz_boardwalk_QGIS\files\good\out\20240401_190203_53_24f2_3B_TOAR_processed_coregistered_res.npz
32610
image_epsg: 32610 type: <class 'int'>
10.242640687119284
2515.9875528840